# 中文数据处理

本python框架目的在于利用python3 对中文数据进行相关处理工作，以此能够较为稳定的处理字符编码与转换。从而方便提取有用信息。方便进行数据分析。

## 数据清理

* 添加索引号，方便进行定位和信息搜索
* 规整位置信息数据-GIS定位准备
* 规整土地价格、面积数据 为计算准备
* 规整容积率数据


In [ ]:
# encoding=utf-8
import pandas as pd
import numpy as np
import sys 
import os
import glob


# import scipy as sp

### 添加索引号



* 重新进行reindex 
* 创建索引指标，目的在于进行匹配时，方便联系一级二级匹配的数据
* 对一、二级市场数据进行贴标签



* 一级市场：标记索引号
* 二级市场：标记索引号

二级市场： 工业 0 开头 01，02... 第一位标记土地类型

一级市场： 第一二位标记土地类型，第二位标记供地方式

土地类型：

* 工业用地-00
* 商业用地-01


供地方式：

* 拍卖出让-01
* 招标出让-02
* 挂牌出让-03




In [ ]:
## 二级市场索引

# reset the index 
print(len(sm_industry))
sm_industry=sm_industry.reset_index(drop=True) # OK 
#sm_industry.index=range(len(sm_industry)) # 可以reset
print(sm_industry.index)

# set the index column 
sm_industry['index']="00"
sm_industry['index']=sm_industry['index']+sm_industry.index.astype(str)

# print(sm_industry['index'].shape)
# sm_industry['index']

In [ ]:
## 一级市场
# reset the index 
print(len(fm_industry))
fm_industry=fm_industry.reset_index(drop=True )  
# fm_industry.reindex(index=range(len(fm_industry)))# 无法reset 
# fm_industry.index=range(len(fm_industry)) # 可以reset
print(fm_industry.index)


# set the index column 
# 土地类型 工业土地为00
fm_industry['index']="00"
# 供地方式
temp=pd.DataFrame(index=fm_industry.index,columns=[0])
temp[fm_industry['供地方式'].str.contains('拍卖出让')]="01"
temp[fm_industry['供地方式'].str.contains('招标出让')]="02"
temp[fm_industry['供地方式'].str.contains('挂牌出让')]="03"

fm_industry['index']=fm_industry['index']+temp[0]

fm_industry['index']=fm_industry['index']+fm_industry.index.astype(str)
print(fm_industry['index'].shape)

## 行政区划处理

In [ ]:
## 二级市场
sm_industry[u'最后一级'].fillna("-",inplace=True)
sm_industry[u'县'].fillna("-",inplace=True)
sm_industry[u'市'].fillna("-",inplace=True)
# tt=data_sm[u'最后一级'].str.replace("-","+")

# data_sm[u'市']
sm_industry[u'最后一级']=sm_industry[u'最后一级'].str.replace("-",data_sm[u'县'].values[0])
sm_industry[u'最后一级']=sm_industry[u'最后一级'].str.replace("-",data_sm[u'市'].values[0])
sm_industry[u'最后一级']=sm_industry[u'最后一级'].str.replace("-",data_sm[u'省'].values[0])
sm_industry[u'县']=sm_industry[u'县'].str.replace("-",'')
sm_industry[u'市']=sm_industry[u'市'].str.replace("-",'')

In [ ]:
## 一级市场

### 将相关数字格式化显示

主要有：

* 成交价格
* 面积
* 容积率
* 日期

In [ ]:
#######################
# 二级市场
#######################

###################
#面积 规整到平方米
###################
dd=sm_industry[sm_industry[u'面积'].str.contains(u"亩")][u'面积'].str.replace(u"亩","").astype(float)*666.67
dd=dd.append(sm_industry[sm_industry[u'面积'].str.contains(u"平方米")][u'面积'].str.replace(u"平方米","").astype(float))
dd=dd.append(sm_industry[sm_industry[u'面积'].str.contains(u"公顷")][u'面积'].str.replace(u"公顷","").astype(float)*10000)
print(dd.shape)
sm_industry[u'面积']=dd


#####################
#价格
#####################

import re
sm_industry=sm_industry[sm_industry[u'转让费'].str.contains(u"万|亿元")]
dd=sm_industry[sm_industry[u'转让费'].str.contains(u"万元")][u'转让费'].str.replace(u"万元",'').str.extract(r'(\d+)')
dd=dd.append(sm_industry[sm_industry[u'转让费'].str.contains(u"亿元")][u'转让费'].str.extract(r'(\d+)').astype(float)*10000)
sm_industry[u'转让费']=dd


####################
#容积率
####################

sm_industry[u'容积率']=sm_industry[sm_industry[u'详细信息'].str.contains(u'容积率')][u'详细信息'].str.extract(r'容积率(\d+\.\d+)')
sm_industry[u'容积率'].fillna(-999,inplace=True)
sm_industry[u'容积率'].astype(str)

In [ ]:
#######################
# 一级级市场  工业用地
#######################

######################
#面积：平方米
######################
fm_industry[u'面积(公顷)']=fm_industry[u'面积(公顷)'].astype(float)*10000


#####################
#价格：万元
#####################
fm_industry['成交价格(万元)']=fm_industry['成交价格(万元)'].astype(float)


#####
# 容积率
#####
fm_industry[u'约定容积率上限'].fillna(-999,inplace=True)
fm_industry[u'约定容积率上限'].astype(float)

#####
# 土地使用年限
#####
fm_industry.loc[fm_industry['土地使用年限'].str.contains("工业"),'土地使用年限']=50

In [ ]:
# 判断 错误数据方法 nan 
data_fm[data_fm[u'土地用途'].apply(np.isreal)]

## 存储数据

In [ ]:
## save the data 
# data_fm.to_excel(path+'data_fm.xlsx','Sheet1')
fm_industry.to_excel(path+'fm_industry.xlsx','Sheet1')
sm_industry.to_excel(path+'sm_industry.xlsx','Sheet1')